# AI4ALL Sessió 3

Llibreries que cal instal·lar (amb conda o pip):
* tqdm
* unidecode

## 1. Xarxes neurals recurrents
Els objectius d'aquest exemple són:
1. Aprendre sobre el funcionament de les xarxes neurals recurrents.
2. Aprendre a utilitzar xarxes pre-entrenades

### 1.1. Dades
En aquest exemple aprendrem una xarxa neural recurrent a predir la següent lletra d'un text a partir de les anteriors. Per exemple:

<img src="char_rnn/charseq.jpeg" style="width: 50%;"/>

Com podeu veure, utilitzant la paraula "hello", per a la lletra "h" volem predir "e". Per a la lletra "e" volem predir "l", i així successivament. Com que les xarxes neurals recurrents tenen memòria, quan predigui la lletra "o", ho farà sabent que anteriorment s'ha entrat "hell".

D'aquesta manera la xarxa apren com funciona la interacció entre lletres per a formar paraules, i és capaç d'utilitzar tot el què ha vist anteriorment per a predir la següent lletra.


### 1.2. Xarxa
L'ideal seria que la xarxa aprenguès a predir la següent lletra enrecordant-se de tot el que s'ha dit abans, **no obstant** els ordinadors no tenen capacitat suficient per a guardar un historial tan gran, i el que fem és definir una finestra de N lletres que anirà lliscant sobre el text. És a dir, la xarxa predirà la següent lletra enrecordant-se de les N lletres anteriors.

Per a aquest exemple he pre-entrenat una xarxa recurrent de dues capes i 256 neurones:

In [1]:
import torch
import torch.nn as nn

class Recurrent(nn.Module):
    def __init__(self, mida_alfabet, n_neurones, n_capes):
        super().__init__()
        self.n_neurones = n_neurones
        self.n_capes = n_capes
        self.mida_alfabet = mida_alfabet
        # GRU es un tipus de xarxa neural recurrent més efectiu que el model basic (RNN)
        self.rnn = nn.GRU(n_neurones, n_neurones, n_capes, batch_first=True, dropout=0.3)
        # Utilitzem una capa de neurones per a convertir una lletra a un vector i vice-versa. Ex. a = (0.4, 0.2, 0.223)
        self.linear_output = nn.Linear(n_neurones, mida_alfabet, bias=False)
          
    def forward(self, x, alfabet, genera_n=64, temperatura=1.0):
        # Primer passem a la xarxa el text inicial
        b, s = x.size()
        y = nn.functional.embedding(x, self.linear_output.weight) # convertim lletres a vectors
        estat = torch.zeros(self.n_capes, 1, self.n_neurones) # inicialitzem l'estat de la xarxa recurrent a zero (no ha vist cap lletra abans)
        out, estat = self.rnn(y, estat) # passem estat inicial i les seqüencies de lletres a la xarxa recurrent
        
        # Utilitzem última lletra com a entrada per a predir la següent lletra. Repetim múltiples cops.
        ret = []
        for i in range(genera_n):
            # Primer convertim a caracter de l'alfabet l'última predicció i l'afegim al text de sortida
            out = out[0,-1,:].contiguous().view(1, self.n_neurones) # agafem última lletra i redimensionem
            out = self.linear_output(out).view(1, 1, self.mida_alfabet) # convertim la sortida de vector a probabilitat de ser certa lletra
            probabilitats = nn.functional.softmax(out * temperatura, dim=2).numpy().ravel() 
            lletra = alfabet[np.random.choice(np.arange(len(alfabet)), p=probabilitats)] # fem una ruleta sobre les mes provables
            ret.append(lletra) # afegim la predicció actual a la sortida
            
            # Convertim l'última lletra del text a posició dins de l'alfabet
            lletra_num = torch.LongTensor([alfabet.index(ret[-1])]).view(1, 1)
            # Convertim de número a vector
            y = nn.functional.embedding(lletra_num, self.linear_output.weight)
            # Passem el vector que representa la sortida anterior com a entrada de la xarxa
            out, estat = self.rnn(y, estat) # y = lletra, estat = estat anterior
     
            
        return "".join(ret)

**Carreguem dades de xarxa pre-entrenada**
En aquest cas l'he entrenat a predir el text de "Don Quijote". Podeu trobar el fitxer d'entrenament a [https://github.com/prlz77/ai4all2018/sessio3/char_rnn](https://github.com/prlz77/ai4all2018/sessio3/char_rnn)

In [2]:
carregat = torch.load("./char_rnn/model1.pth", map_location="cpu") # llegim fitxer guardat.
""" En el fitxer tambe hi he guardat la correspondencia entre les lletres 
    acceptades i un nombre enter: a = 1, b = 2, etc. Això és necessari 
    perquè la xarxa treballa amb números.
"""
alfabet = carregat["alphabet"] 

Ara carreguem els pesos pre-entrenats a una xarxa recent creada. **L'arquitectura de la xarxa carregada i el seu recipient han de coincidir.**

In [3]:
rnn = Recurrent(len(alfabet), 256, 2) # Creem xarxa inicial sense entrenar
rnn.load_state_dict(carregat["net"]) # AQUI ES ON CARREGUEM XARXA PRE-ENTRENADA
rnn = rnn.cpu() 

### 1.3. Predicció
Finalment podem utilitzar la xarxa apresa amb *Don Quijote* per a generar text. Com? En comptes de donar nosaltres tota l'estona les lletres a l'entrada de la xarxa, li donem una petita entrada inicial i **fem que la xarxa utilitzi la seva propia sortida com a entrada.**

Primer definim unes funcions auxiliars per a convertir el format les entrades i sortides:

In [1]:
from char_rnn.entrena_rnn import preprocess
import numpy as np

# Mida_seq és la mida màxima que la nostra xarxa mira enrere
def text_a_xarxa(text, alfabet):
    text = preprocess(text)
    return torch.LongTensor(list(map(lambda x: alfabet.index(x), text))).view(1, len(text))

def xarxa_a_text(sortida, alfabet, temperatura=1.0):
    probabilitats = nn.functional.softmax(sortida * temperatura, dim=2).numpy()
    b, s, p = probabilitats.shape
    probabilitats = probabilitats.reshape((b*s, p))
    return "".join([alfabet[np.random.choice(np.arange(len(alfabet)), p=x)] for x in probabilitats])


def prediu_text(entrada, quant=300, temperatura=1):
    rnn.eval()
    with torch.no_grad():
        entrada_xarxa = text_a_xarxa(entrada, alfabet)
        return rnn(entrada_xarxa, alfabet, genera_n=quant, temperatura=temperatura)

Passem un text inicial a la xarxa, i la fem continuar des d'allà.

In [5]:
text_inicial = "Sancho, dijo don quijote. Te gusta el vin"
print(text_inicial + prediu_text(text_inicial, quant=600, temperatura=2))

Sancho, dijo don quijote. Te gusta el vino del mal se le habeis traido a todos los caballeros andantes y amparen y cardenio y como a su amo la proceder de la guerra en las cosas de la venta, porque le tocaba a los de pensamientos y se cuentan a la porfia, con todo esto, sancho, porque no se de la tal vez se ha de ser que el despecho de contento de memoria de las cosas que de la procura la cabeza a la mano antes que de la caballeria en la hermosa doncella de su amo, de quien el paso en este caballero de la compania de caballero, mas sabra el rico caballero atado de la mancha, que este es la que habia de ser mejor decir que se entrara 


## Exercicis
1. Canvieu el text inicial i observeu els canvis a la sortida.
2. Quin és l'efecte de la temperatura?
3. Què passa si executeu la predicció múltiples cops? Per què?
3. Quin és l'efecte del paràmetre quant?
4. Canvieu el model per model2.pth. Quins canvis observeu?